In [13]:
import datetime
import time
import re
import string
import pygtrie
from collections import Counter
import collections

In [14]:
#Opens the corpus file to read data
f = open("radikal.corpus","r")
lines = f.readlines()
f.close()

#a dummy counter for counting the number of documents in each iteration
docCounter = 0

#dummy lists for storing instatnt terms in each document
listTerms= []
listTermCounter = []

#dictionaties for storing the information about terms
termDocs = dict()                                          #stores the number of documents that a term existed in
termData = dict()                                          #stores the number of terms in the whole dataset.
counterDict= dict()                                        #

#opens new files to write new data
fD = open("r.docs", "w+")
fT = open("r.terms", "w+")
fDV = open("r.Doc_Vec", "w+")
fIDV = open("r.IDV", "w+")

#iterates through the whole dataset once to split the dataset according to our assignment.
for line in lines:
    #in order to store the name and genre of the document easily they retrieved at the beginning
    if line[:1] == "<":
        newLine= re.sub( '<', '', line)
        newLine= re.sub( '>', '', newLine)
        
        #gets the name of the document and normalizes it
        if newLine.split("=")[0] == "INSTANCE":
            docName = newLine.split("=")[1]
            docName = re.sub( '\n', '', docName)
            docName = re.sub('.html', '', docName)
        
        #get the genre of the document
        elif newLine.split("=")[0] == "CATEGORY":
            docGenre = newLine.split("=")[1]
            docGenre = re.sub( '\n', '', docGenre)
    
    #iterates through the lines which stores the content of the document
    if not line[:1] == "<":
        line= re.sub( '\n', '', line)                  #normalizes the line in the most simple way
        line= re.sub( '  ', ' ', line)                 #normalizes the line as well
        parsedLine = line.split(" ")                   #parses the line according to space
        length = len(parsedLine)
        termCounter = 0
        for i in range(length):
            listTerms.append(parsedLine[i])
            if parsedLine[i] in termData:
                if not parsedLine[i] == ' ' :
                    termData[parsedLine[i]] = termData[parsedLine[i]]+1
                    if not parsedLine[i] in listTermCounter:
                        listTermCounter.append(parsedLine[i])
            else:
                if not parsedLine[i] == ' ':
                    termCounter =termCounter+1
                    termData[parsedLine[i]] = 1
                    if not parsedLine[i] in listTermCounter:
                        listTermCounter.append(parsedLine[i])
        
    #The operations when the all the lines in document finished         
    if line == "</INSTANCE>\n":
        fD.write(str(docCounter)+"\t"+docName + "\t" + docGenre +" \t" + str(len(listTerms)) + "\t" + str(len(Counter(listTerms))) + "\n")
        for k in range(len(listTermCounter)):
            if listTermCounter[k] in termDocs:
                termDocs[listTermCounter[k]] = termDocs[listTermCounter[k]] +1
            else:
                termDocs[listTermCounter[k]] = 1
        counterDict.update({docCounter: dict(Counter(listTerms))})
        docCounter = docCounter +1
        listTerms.clear()
        listTermCounter.clear()


In [15]:
fI = open("r.info", "w+")
fI.write(str(docCounter) + "\t" + str(len(termData))+"\n")
termsSorted= dict()
sortedTerms= dict()
j=0
for term in sorted(termData.keys()):
    termsSorted[str(j)] = term
    sortedTerms[term] = str(j)
    fT.write(str(j) +"\t" + str(termData[term]) +"\t"+ str(termDocs[term])+"\n")
    j=j+1

In [16]:
termInDocs = dict()
for i in range(len(counterDict)):
    for keys in sorted(counterDict[i].keys()):
        #fDV.write(str(keys) +"\t"+str(counterDict[i][keys])+"\t")
        fDV.write(str(sortedTerms[keys])+"\t"+ str(counterDict[i][keys])+"\t")
        if not keys in termInDocs.keys():
            termInDocs[keys] = [str(i)]
        else:
            termInDocs[keys].append(str(i))           
    fDV.write("\n")   
    

In [17]:
for keys in sorted(termInDocs.keys()):
    for i in range(len(termInDocs[keys])):
        fIDV.write(str(termInDocs[keys][i])+"\t")
    fIDV.write("\n")    